In [ ]:
import networkx as nx
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator

print("Loading P7_rolling_ridge.qasm...")
qc = QuantumCircuit.from_qasm_file("P7_rolling_ridge.qasm")

# Identify the Two Groups (Graph Theory)
print("Splitting the circuit...")
qubit_map = {bit: i for i, bit in enumerate(qc.qubits)}
G = nx.Graph()
G.add_nodes_from(range(qc.num_qubits))

# Build graph of interactions
for instruction in qc.data:
    op, qargs, cargs = instruction
    if len(qargs) > 1:
        indices = [qubit_map[q] for q in qargs]
        for i in range(len(indices)):
            for j in range(i + 1, len(indices)):
                G.add_edge(indices[i], indices[j])

components = list(nx.connected_components(G))
groups = [sorted(list(g)) for g in components]

# Simulate Each Group Separately
final_bit_array = ['0'] * qc.num_qubits
sim = AerSimulator(method='statevector')

print(f"Solving {len(groups)} independent parts...")

for group_idx, group_qubits in enumerate(groups):
    # Create a mini-circuit
    sub_qc = QuantumCircuit(len(group_qubits))

    # Map original indices to 0, 1, 2... for the sub-circuit
    old_to_new = {old_idx: new_idx for new_idx, old_idx in enumerate(group_qubits)}

    for instruction in qc.data:
        op = instruction.operation
        qargs = instruction.qubits

        # Check if this gate belongs to the current group
        if qubit_map[qargs[0]] in group_qubits:
            # Map the qubits to the new small circuit
            new_qargs = [sub_qc.qubits[old_to_new[qubit_map[q]]] for q in qargs]

            # Don't copy existing measurements
            if op.name != 'measure':
                sub_qc.append(op, new_qargs)

    # ADD MEASUREMENTS NOW (After gates are added)
    sub_qc.measure_all()

    print(f"  - Simulating Chunk {group_idx+1} ({len(group_qubits)} qubits)...")
    job = sim.run(sub_qc, shots=1024)
    result = job.result()
    counts = result.get_counts()
    winner_sub = max(counts, key=counts.get)
    bits_ordered = list(reversed(winner_sub))

    for i, bit_val in enumerate(bits_ordered):
        original_index = group_qubits[i]
        final_bit_array[original_index] = bit_val

solution = "".join(reversed(final_bit_array))

print(f"\nSUCCESS!")
print(f"Solution Bitstring: {solution}")

Loading P7_rolling_ridge.qasm...
Splitting the circuit...
Solving 2 independent parts...
  - Simulating Chunk 1 (26 qubits)...


/tmp/ipython-input-1335300782.py:17: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 3.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  op, qargs, cargs = instruction


  - Simulating Chunk 2 (20 qubits)...

SUCCESS!
Solution Bitstring: 0110001011111111110011110001000010011011100000
